In [ ]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/')

In [ ]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

In [ ]:
import scipy.stats as st
from scipy.fftpack import fft, fftfreq 
from scipy.signal import argrelextrema
import operator

In [ ]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [ ]:
def mean_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
    average = np.mean(sample[:,col])
    feat.append(average)

  return feat

def max_ts(sample, Te=1.0):
   
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amax(sample[:,col])
      feat.append(average)

  return feat

def min_ts(sample, Te=1.0): 
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amin(sample[:,col]) # min
      feat.append(average)

  return feat 

def skew_ts(sample, Te=1.0):    
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.skew(sample[:,col]) # skew
      feat.append(average)

  return feat

def kurtosis_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.kurtosis(sample[:,col]) # kurtosis 
      feat.append(average)

  return feat
 
def iqr_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.iqr(sample[:,col]) # interquartile rante
      feat.append(average)

  return feat

def mad_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.median(np.sort(abs(sample[:,col] - np.median(sample[:,col]))))
      feat.append(average)

  return feat

def area_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col], dx=Te)
      feat.append(average)

  return feat                            
 

def sq_area_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col] ** 2, dx=Te)
      feat.append(average)

  return feat
def energy_measure(sample):
  """Calculates energy measures"""
  feat = []
  for col in range(0,sample.shape[1]):

    em_x = np.mean(np.square(sample[:,col] ))
    feat.append(em_x)
  return feat  





In [ ]:
def stat_area_features(x, Te=1.0):

    mean_ts = np.mean(x, axis=1).reshape(-1,1) # mean
    max_ts = np.amax(x, axis=1).reshape(-1,1) # max
    min_ts = np.amin(x, axis=1).reshape(-1,1) # min
    std_ts = np.std(x, axis=1).reshape(-1,1) # std
    skew_ts = st.skew(x, axis=1).reshape(-1,1) # skew
    kurtosis_ts = st.kurtosis(x, axis=1).reshape(-1,1) # kurtosis 
    iqr_ts = st.iqr(x, axis=1).reshape(-1,1) # interquartile rante
    mad_ts = np.median(np.sort(abs(x - np.median(x, axis=1).reshape(-1,1)),
                               axis=1), axis=1).reshape(-1,1) # median absolute deviation
    area_ts = np.trapz(x, axis=1, dx=Te).reshape(-1,1) # area under curve
    sq_area_ts = np.trapz(x ** 2, axis=1, dx=Te).reshape(-1,1) # area under curve ** 2

    return np.concatenate((mean_ts,max_ts,min_ts,std_ts,skew_ts,kurtosis_ts,
                           iqr_ts,mad_ts,area_ts,sq_area_ts), axis=1)
def frequency_domain_features(x, Te=1.0):
  feat=[]

  # As the DFT coefficients and their corresponding frequencies are symetrical arrays
  # with respect to the middle of the array we need to know if the number of readings 
  # in x is even or odd to then split the arrays...
  if x.shape[1]%2 == 0:
      N = int(x.shape[1]/2)
  else:
      N = int(x.shape[1]/2) - 1
  xf = np.repeat(fftfreq(x.shape[1],d=Te)[:N].reshape(1,-1), x.shape[0], axis=0) # frequencies
  dft = np.abs(fft(x, axis=1))[:,:N] # DFT coefficients   
  
  # statistical and area features
  # dft_features = stat_area_features(dft, Te=1.0)
  # weighted mean frequency
  dft_weighted_mean_f = np.average(xf, axis=1, weights=dft).reshape(-1,1)
  # 5 first DFT coefficients 
  dft_first_coef = dft[:,:5]    
  # 5 first local maxima of DFT coefficients and their corresponding frequencies
  dft_max_coef = np.zeros((x.shape[0],5))
  dft_max_coef_f = np.zeros((x.shape[0],5))
  for row in range(x.shape[0]):
      # finds all local maximas indexes
      extrema_ind = argrelextrema(dft[row,:], np.greater, axis=0) 
      # makes a list of tuples (DFT_i, f_i) of all the local maxima
      # and keeps the 5 biggest...
      extrema_row = sorted([(dft[row,:][j],xf[row,j]) for j in extrema_ind[0]],
                            key=operator.itemgetter(0), reverse=True)[:5] 
      for i, ext in enumerate(extrema_row):
          dft_max_coef[row,i] = ext[0]
          dft_max_coef_f[row,i] = ext[1]  

  feat.append(np.concatenate((dft_weighted_mean_f,dft_first_coef, dft_max_coef,dft_max_coef_f), axis=1).tolist() )
  # print(len(feat))
  # print(feat)
  return feat

In [ ]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks.tolist())

    return feat

In [ ]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        features = np.hstack((features, mean_ts(sample)))
        features = np.hstack((features, max_ts(sample)))
        features = np.hstack((features, min_ts(sample)))
        features = np.hstack((features, kurtosis_ts(sample)))
        features = np.hstack((features, iqr_ts(sample)))
        features = np.hstack((features, skew_ts(sample)))
        features = np.hstack((features, area_ts(sample)))
        features = np.hstack((features, sq_area_ts(sample)))
        features = np.hstack((features, mad_ts(sample)))
        features = np.hstack((features, mean_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, max_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, min_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, kurtosis_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, iqr_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, skew_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, area_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, sq_area_ts(sample[:,1:]-sample[:,:-1],1)))
        features = np.hstack((features, mad_ts(sample[:,1:]-sample[:,:-1],1)))
       
        X_tmp.append(features)
       
    X = np.array(X_tmp)
    
    return X

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
def Model(num_classes):
  layers = [
            

      tf.keras.layers.Dense(128, activation =tf.nn.leaky_relu),
     
      tf.keras.layers.Dense(64, activation =tf.nn.leaky_relu),
      
      tf.keras.layers.Dense(32, activation = tf.nn.leaky_relu),
     
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  X_train, X_test, y_train, y_test=train_test_split( X, Y, test_size=0.33, random_state=42)
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
 
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
 


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
 

  model = Model(num_classes)
  es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=100)
  

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, y_train, batch_size=64, epochs=250)
  _,acc=model.evaluate(X_test,y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [ ]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [ ]:
def TrainAndReportAccuracyOpportunity(X_train,Y_train,X_test,Y_test,num_classes):
  avg_acc = []
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, Y_train.shape)
  print(X_test.shape,Y_test.shape)

  model = Model(num_classes)
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, Y_train, batch_size=32, epochs=250)
  _,acc=model.evaluate(X_test,Y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))

In [ ]:
def OpportunityDataSetAnalysis():
  X_train=np.load('data/Opportunity/Opportunity_train_X.npz')['arr_0']
  X_test=np.load('data/Opportunity/Opportunity_test_X.npz')['arr_0']
  Y_train=np.load('data/Opportunity/Opportunity_train_y.npz')['arr_0']
  Y_test=np.load('data/Opportunity/Opportunity_test_Y.npz')['arr_0']
  classes_number = Y_train.shape[1]
  Y_train = np.argmax(Y_train, axis=1)
  Y_test = np.argmax(Y_test, axis=1)
  return X_train,Y_train,X_test,Y_test,classes_number

In [ ]:
X_train,Y_train,X_test,Y_test,num_classes=OpportunityDataSetAnalysis()
  

In [ ]:
X_train = feature_extraction(X_train)
X_test = feature_extraction(X_test)

In [ ]:
TrainAndReportAccuracyOpportunity(X_train,Y_train,X_test,Y_test,num_classes)

In [ ]:
RunOpportunity()

In [ ]:
Run('data/LOSO/MHEALTH.npz')

In [ ]:
Run('data/LOTO/MHEALTH.npz')

In [ ]:
Run('data/FNOW/MHEALTH.npz')

In [ ]:
Run('data/SNOW/MHEALTH.npz')

In [ ]:
Run('data/LOTO/USCHAD.npz')

In [ ]:
Run('data/LOSO/USCHAD.npz')

In [ ]:
Run('data/FNOW/USCHAD.npz')

In [ ]:
Run('data/SNOW/USCHAD.npz')

In [ ]:
Run('data/LOTO/WHARF.npz')

In [ ]:
Run('data/LOSO/WHARF.npz')

In [ ]:
Run('data/SNOW/WHARF.npz')

In [ ]:
Run('data/FNOW/WHARF.npz')

In [ ]:
Run('data/LOTO/WISDM.npz')

In [ ]:
Run('data/LOSO/WISDM.npz')

In [ ]:
Run('data/SNOW/WISDM.npz')

In [ ]:
Run('data/FNOW/WISDM.npz')

In [ ]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

In [ ]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

In [ ]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

In [ ]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

In [ ]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

In [ ]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

In [ ]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

In [ ]:
Run('data/FNOW/UTD-MHAD1_1s.npz')